# Applied Machine Learning homework3

Huibo Zhao hz2480
Lingjie Xu lx2222

# Perform General Data Preprocessing

In [57]:
import os
import pandas as pd
import math
import numpy as np
import random
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, LassoCV

In [58]:
# Extract those files that contain our training data
path = os.getcwd()
files = os.listdir(path)
xlsx_files = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~' and f[0:4] != '2018']

# Putting all the contents into dataframe for viewing and accessing
df = pd.DataFrame()
for f in xlsx_files:
    data = pd.read_excel(f, 'FEguide')
    df = df.append(data)

In [59]:
train_row_count = df.shape[0]
train_row_count

3701

In [60]:
xlsx_test = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~' and f[0:4] == '2018']
xlsx_test

['2018 FE Guide for DOE-release dates before 2-24-2018-no-sales-2-23-2018public.xlsx']

In [61]:
test_data = pd.read_excel(xlsx_test[0], 'FEguide')
df = df.append(test_data)

In [62]:
df.shape

(4928, 162)

In [63]:
# We extract the target value
y = df['Comb Unrd Adj FE - Conventional Fuel'].as_matrix()

# Examine if the target contains any empty or nan value
# If it does, we need to drop the row without a valid target value
# Fortunately, we don't find such row
for i in y:
    if isinstance(i,float) and math.isnan(i):
        print(i)

In [64]:
dropout_features = list()


# Professor had named a few features that we cannot use, we extract the name of those features
da = np.array(df.columns)
drop_index = list()
for i in range(len(da)):
    if "FE" in da[i]:
        drop_index.append(da[i])
    elif "MPG" in da[i]:
        drop_index.append(da[i])
    elif "CO2" in da[i]:
        drop_index.append(da[i])
    elif "Smog" in da[i]:
        drop_index.append(da[i])
    elif "Guzzler" in da[i]:
        drop_index.append(da[i])
    elif "EPA" in da[i]:
        drop_index.append(da[i])
    elif "GHG" in da[i]:
        drop_index.append(da[i])

dropout_features.extend(drop_index)
df_1 = df.drop(drop_index, axis=1) ##########first drop : drop disallowed features##########
############################################################################################

# We examine each feature and count how many nan values it have
# Save results in a dictionary

column_names = df_1.columns.values
nan_dict = {}

for column_name in column_names:
    count = 0
    for i in range(0,df_1.shape[0]):
        if isinstance(df_1.iloc[i][column_name],float) and math.isnan(df_1.iloc[i][column_name]):
            count += 1
    nan_dict[column_name] = count
    
drop_2 = list()
for key in nan_dict:
    if nan_dict[key]>=2000:
        drop_2.append(key)

dropout_features.extend(drop_2)
df_2 = df_1.drop(drop_2, axis=1) ##########second drop : drop features without enough valid entry#########
##########################################################################################################

In [65]:
print(nan_dict)

{'Model Year': 0, 'Mfr Name': 0, 'Division': 0, 'Carline': 0, 'Verify Mfr Cd': 0, 'Index (Model Type Index)': 0, 'Eng Displ': 0, '# Cyl': 0, 'Transmission': 0, 'Air Aspir Method': 2522, 'Air Aspiration Method Desc': 0, 'Trans': 0, 'Trans Desc': 0, 'Trans, Other': 4904, '# Gears': 0, 'Lockup Torque Converter': 0, 'Trans Creeper Gear': 0, 'Drive Sys': 0, 'Drive Desc': 0, 'Max Ethanol % - Gasoline': 107, 'Max Biodiesel %': 4822, 'Range1 - Model Type Driving Range - Conventional Fuel': 4640, 'Fuel Usage  - Conventional Fuel': 0, 'Fuel Usage Desc - Conventional Fuel': 0, 'Fuel Unit - Conventional Fuel': 0, 'Fuel Unit Desc - Conventional Fuel': 0, '2Dr Pass Vol': 4160, '2Dr Lugg Vol': 4160, '4Dr Pass Vol': 3172, '4Dr Lugg Vol': 3172, 'Htchbk Pass Vol': 4481, 'Htchbk Lugg Vol': 4481, 'Annual Fuel1 Cost - Conventional Fuel': 0, ' Range2 - Alt Fuel Model Typ Driving Range - Alternative Fuel': 4641, ' Fuel2 Usage - Alternative Fuel': 4641, ' Fuel2 Usage Desc - Alternative Fuel': 4641, 'Fuel2 Uni

In [66]:
drop_2

['Air Aspir Method',
 'Trans, Other',
 'Max Biodiesel %',
 'Range1 - Model Type Driving Range - Conventional Fuel',
 '2Dr Pass Vol',
 '2Dr Lugg Vol',
 '4Dr Pass Vol',
 '4Dr Lugg Vol',
 'Htchbk Pass Vol',
 'Htchbk Lugg Vol',
 ' Range2 - Alt Fuel Model Typ Driving Range - Alternative Fuel',
 ' Fuel2 Usage - Alternative Fuel',
 ' Fuel2 Usage Desc - Alternative Fuel',
 'Fuel2 Unit - Alternative Fuel',
 'Fuel2 Unit Desc - Alternative Fuel',
 'Fuel2 Annual Fuel Cost - Alternative Fuel',
 'Unnamed: 74',
 'Relabel',
 'Relabel Desc',
 'Comments - Mfr Eng Cnfg',
 'Cyl Deact Desc',
 'Var Valve Lift Desc',
 'Energy Storage Device Desc',
 'Energy Storage Device,If Other',
 '# Batteries',
 'Battery Type Desc',
 'Battery Type, If Other',
 'Total Voltage for Battery Pack(s)',
 'Batt Energy Capacity (Amp-hrs)',
 'Batt Specific Energy (Watt-hr/kg)',
 'Batt Charger Type Desc',
 'Comments',
 '# Capacitors',
 'Regen Braking Type Desc',
 'Regen Braking Type, If Other',
 'Regen Braking Wheels Source (Front, 

In [67]:
len(dropout_features)

116

In [68]:
# After the second drop, we left with 48 features
# We found 8 features have at least 1 nan values  
# We have to handle that 
count_dict = {}
column_names = df_2.columns.values
for column_name in column_names:
    count = 0
    for i in range(0,df_2.shape[0]):
        if isinstance(df_2.iloc[i][column_name],float) and math.isnan(df_2.iloc[i][column_name]):
            count += 1
    count_dict[column_name] = count
    
        
for key in count_dict:
    if count_dict[key] > 0:
        print(str(key) + ": " + str(count_dict[key]))


Max Ethanol % - Gasoline: 107
Descriptor - Model Type (40 Char or less): 1391
Car/Truck Category - Cash for Clunkers Bill.: 1690
Unique Label?: 420
Label Recalc?: 248
Var Valve Timing Desc: 151
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) : 1301


In [69]:
dropout_features.append("Car/Truck Category - Cash for Clunkers Bill.")
df_3 = df_2.drop("Car/Truck Category - Cash for Clunkers Bill.", axis=1) #### third drop ####
####### Car/Truck features is useless after examining it ####################################

dropout_features.append("Release Date") ###### Fourth drop ##################################
df_4 = df_3.drop("Release Date", axis=1) ##### We temporarily drop the date here ############

# dropout_features.append("Air Aspir Method")
dropout_features.append("Descriptor - Model Type (40 Char or less)")
# df_5 = df_4.drop(["Air Aspir Method", "Descriptor - Model Type (40 Char or less)"], axis=1)
df_5 = df_4.drop("Descriptor - Model Type (40 Char or less)", axis=1)

#################### We have done all dropping features by this point #######################
#################### All dropout features are stored in dropout_features ####################


In [70]:
df_5.shape

(4928, 43)

In [71]:
################################ We fill out those four features manually ###################
index_ = df_5.columns
adjust_index = list()
adjust_index.append(list(index_).index("Max Ethanol % - Gasoline"))
adjust_index.append(list(index_).index("Unique Label?"))
adjust_index.append(list(index_).index("Label Recalc?"))
adjust_index.append(list(index_).index("Var Valve Timing Desc"))
adjust_index

for i in adjust_index:
    for j in range(df_5.shape[0]):
        if j != df_5.shape[0] - 1:
            if isinstance(df_5.iat[j, i], float) and math.isnan(df_5.iat[j, i]):
                if isinstance(df_5.iat[j + 1, i], float) and math.isnan(df_5.iat[j + 1, i]):
                    df_5.iat[j, i] = df_5.iat[j - 1, i]
                else:
                    m = random.randint(0,1)
                    if m == 0:
                        df_5.iat[j, i] = df_5.iat[j - 1, i]
                    else:
                        df_5.iat[j, i] = df_5.iat[j + 1, i]
        else:
            if isinstance(df_5.iat[j, i],float) and math.isnan(df_5.iat[j, i]):
                df_5.iat[j, i] = df_5.iat[j - 1, i]
##############################################################################################

############################# using get_dummies to fill out discrete features ################
df_6 = pd.get_dummies(df_5)
##############################################################################################

In [72]:
import fancyimpute

############################# using fancy_impute to impute data entries ######################
X = df_6.as_matrix()
mice = fancyimpute.MICE(verbose=0)
X_fancy_mice = mice.complete(X)
#############################################################################################

# Split
# X_train, X_val, y_train, y_val = train_test_split(
#     X_fancy_mice, y, random_state=0)



In [73]:
X_fancy_mice.shape

(4928, 1766)

In [77]:
y.shape

(4928,)

In [79]:
X_15_17 = X_fancy_mice[0:train_row_count, :]
X_18 = X_fancy_mice[train_row_count: , :]
y_15_17 = y[0:train_row_count,]
y_18 = y[train_row_count:,]

In [83]:
X_15_17.shape

(3701, 1766)

In [9]:
# Evaluate
# ridge_pipe = make_pipeline(StandardScaler(), RidgeCV())
# scores = cross_val_score(ridge_pipe, X_fancy_mice, y, cv=10)
# np.mean(scores), np.std(scores)

In [84]:
lasso_pipe = make_pipeline(StandardScaler(), LassoCV())
lasso_pipe.fit(X_15_17, y_15_17)
lasso_pipe.score(X_18, y_18)
# scores = cross_val_score(lasso_pipe, X_fancy_mice, y, cv=10)
# np.mean(scores), np.std(scores)

0.83124557772154573

In [33]:
xlsx_test = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~' and f[0:4] == '2018']

In [34]:
xlsx_test

['2018 FE Guide for DOE-release dates before 2-24-2018-no-sales-2-23-2018public.xlsx']

In [35]:
test_data = pd.read_excel(xlsx_test[0], 'FEguide')
df_test = pd.DataFrame()
df_test = df_test.append(test_data)

In [36]:
df_test

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,City CO2 Rounded Adjusted,Hwy CO2 Rounded Adjusted,Comb CO2 Rounded Adjusted (as shown on FE Label),CO2-PHEV Composite Rounded Adjusted Combined CO2,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe
0,2018,Honda,Acura,NSX,HNX,57,3.5,6,Auto(AM-S9),21,...,419,406,420,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018,FCA US LLC,ALFA ROMEO,4C,CRX,410,1.8,4,Auto(AM6),24,...,365,259,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018,Volkswagen Group of,Audi,R8 AWD,VGA,65,5.2,10,Auto(AM-S7),14,...,634,406,531,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018,Volkswagen Group of,Audi,R8 RWD,VGA,71,5.2,10,Auto(AM-S7),14,...,612,358,497,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018,Volkswagen Group of,Audi,R8 Spyder AWD,VGA,66,5.2,10,Auto(AM-S7),14,...,634,406,531,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2018,Volkswagen Group of,Audi,R8 Spyder RWD,VGA,72,5.2,10,Auto(AM-S7),14,...,612,358,497,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2018,Volkswagen Group of,Audi,TT Roadster quattro,VGA,46,2.0,4,Auto(AM-S6),23,...,373,290,336,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2018,BMW,BMW,M4 DTM Champions Edition,BMX,488,3.0,6,Auto(AM-S7),17,...,513,373,450,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2018,Volkswagen Group of,Bugatti,Chiron,VGA,38,8.0,16,Auto(AM-S7),9,...,1010,632,840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2018,General Motors,Chevrolet,CORVETTE,GMX,278,6.2,8,Auto(S8),15,...,594,356,487,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df_test_1 = df_test.drop(dropout_features, axis=1)

In [38]:
len(df_test_1.columns)

43

In [39]:
df_test_1.to_csv('test.csv')

In [40]:
# After the second drop, we left with 48 features
# We found 8 features have at least 1 nan values  
# We have to handle that 
count_dict = {}
column_names = df_test_1.columns.values
for column_name in column_names:
    count = 0
    for i in range(0,df_test_1.shape[0]):
        if isinstance(df_test_1.iloc[i][column_name],float) and math.isnan(df_test_1.iloc[i][column_name]):
            count += 1
    count_dict[column_name] = count
    
        
for key in count_dict:
    if count_dict[key] > 0:
        print(str(key) + ": " + str(count_dict[key]))

Max Ethanol % - Gasoline: 27
Unique Label?: 125
Label Recalc?: 61
Var Valve Timing Desc: 44
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) : 221


In [41]:
################################ We fill out those four features manually ###################
index_ = df_test_1.columns
adjust_index = list()
adjust_index.append(list(index_).index("Max Ethanol % - Gasoline"))
adjust_index.append(list(index_).index("Unique Label?"))
adjust_index.append(list(index_).index("Label Recalc?"))
adjust_index.append(list(index_).index("Var Valve Timing Desc"))
adjust_index

for i in adjust_index:
    for j in range(df_test_1.shape[0]):
        if j != df_test_1.shape[0] - 1:
            if isinstance(df_test_1.iat[j, i], float) and math.isnan(df_test_1.iat[j, i]):
                if isinstance(df_test_1.iat[j + 1, i], float) and math.isnan(df_test_1.iat[j + 1, i]):
                    df_test_1.iat[j, i] = df_test_1.iat[j - 1, i]
                else:
                    m = random.randint(0,1)
                    if m == 0:
                        df_test_1.iat[j, i] = df_test_1.iat[j - 1, i]
                    else:
                        df_test_1.iat[j, i] = df_test_1.iat[j + 1, i]
        else:
            if isinstance(df_test_1.iat[j, i],float) and math.isnan(df_test_1.iat[j, i]):
                df_test_1.iat[j, i] = df_test_1.iat[j - 1, i]
##############################################################################################

In [43]:
df_test_1.shape

(1227, 43)

In [45]:
df_5.shape

(3701, 43)

In [48]:
df_whole = pd.DataFrame()
df_whole = df_whole.append(df_5)
df_whole = df_whole.append(df_test_1)
df_whole_1 = pd.get_dummies(df_whole)
df_whole_1.shape

(4928, 1766)

In [ ]:
df_whole_1 = df_whole.get_

In [24]:
X_test = df_test_2.as_matrix()
X_test_fancy_mice = mice.complete(X_test)

In [25]:
y_test = df_test['Comb Unrd Adj FE - Conventional Fuel'].as_matrix()

In [31]:
lasso_pipe.score(X_test_fancy_mice, y_test)

ValueError: operands could not be broadcast together with shapes (1227,1190) (1612,) (1227,1190) 

In [32]:
X_test_fancy_mice.shape

(1227, 1190)

In [ ]:
y_test.shape